# Classification Model
* File Name: Reference_implementation of BFSI 
* Date of creation(dd-mm-yyyy) : 27-01-2023
* Author Name/Dept : AIShield
* Organization : BGSW
* Description : Source Code of reference implementation of AIShield API
* Copyright : Copyright 2022 Bosch Global Software Technologies Private Limited. All Rights Reserved.

### Metadata
* Dataset: Banking Marketing Campagin
* Size of dataset:41188, 18
* Number of class : 2
* Original Model: XGBOOST 

### Outcomes
* Accuracy of model: 0.92


In [71]:
"""
Description: commands to install all the packages, remove comments to install all the libraries
"""
# ! pip install xgboost==1.6.2
# ! pip install pandas==1.5.1
# ! pip install scikit-learn==1.1.3
# ! pip install numpy==1.23.4
# ! pip install pyminizip
# !pip install requests==2.28.0
# !pip install humanfriendly==9.2

'\nDescription: commands to install all the packages, remove comments to install all the libraries\n'

# 1.0 Importing libraries

In [1]:
"""
Description: Import libraries
"""

import os
import copy
import json
import time
import pickle
import shutil
import requests
import py_compile
import pyminizip
import zipfile
import numpy as np
import pandas as pd
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

# 2.0 Data Loading and Preprocessing

#### Download dataset from the link https://archive.ics.uci.edu/ml/machine-learning-databases/00222/  
1. then download the bank-additional.zip file and extract it

In [2]:
'''
Description : Loading credit card fraud detection dataset
'''
df = pd.read_csv('bank-additional/bank-additional-full.csv',delimiter=";")

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
df.shape

(41188, 21)

In [5]:
df.replace(['basic.6y','basic.4y', 'basic.9y'], 'basic', inplace=True)

In [6]:
columns = ['day_of_week','month']
df.drop(columns, axis=1, inplace=True)

In [7]:
'''Description : Label Encoding the Categorical Columns'''

le =LabelEncoder()
df.job = le.fit_transform(df.job)
df.marital = le.fit_transform(df.marital)
df.education = le.fit_transform(df.education)
df.housing = le.fit_transform(df.housing)
df.loan = le.fit_transform(df.loan)
df.poutcome = le.fit_transform(df.poutcome)
df.contact = le.fit_transform(df.contact)
df.default = le.fit_transform(df.default)
df.y = le.fit_transform(df.y)

In [8]:
df.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,261,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,1,1,0,0,1,149,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,1,0,2,0,1,226,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,0,0,0,0,1,151,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,1,0,0,2,1,307,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [9]:
output = 'y'
 
X = df.loc[:, df.columns != output]
y = df['y']

In [10]:
'''
Description : Splitting data for validation
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [11]:
'''
Description : Check size of dataset
'''
print("shape of x_train: ",X_train.shape)
print("shape of y_train: {}".format(y_train.shape))
print(f'shape of x_test: {X_test.shape}')
print(f'shape of y_test: {y_test.shape}')

shape of x_train:  (30891, 18)
shape of y_train: (30891,)
shape of x_test: (10297, 18)
shape of y_test: (10297,)


# 3.0 Model Development and Training

In [12]:
def make_directory(directory):
    """
    create directory

    Parameters
    ----------
    directorys : list containing the directorys path to create 
    Returns
    -------
    None.

    """
    for d in directory:
        if os.path.isdir(d):
            print("directory {} already exist".format(d))
        if os.path.isdir(d)==False:
            os.mkdir(path=d)
            print("directory {} created successfully".format(d))

In [13]:
def delete_directory(directorys):
    """
    delete directory 

    Parameters
    ----------
    directorys : list containing the directorys to deleate along with all the files

    Returns
    -------
    None.

    """
    if len(directorys)>=1:
        for d in directorys:
            if os.path.isdir(d):
                try:
                    if os.path.isfile(d):
                        os.remove(path=d)
                    else:
                        shutil.rmtree(path=d)
                        print("Removed: {}".format(d))
                except:
                    print("Failed to removed: {}".format(d))
                

In [14]:
def make_archive(base_name,root_dir,zip_format='zip'):
    """
    created zip for given folder

    Parameters
    ----------
    base_name : name of zip file
    root_dir : directory to archive/zip
    zip_format : zip or tar 
        DESCRIPTION. The default is 'zip'.

    Returns
    -------
    None.

    """
    shutil.make_archive(base_name=base_name, format=zip_format, root_dir=root_dir)
    

In [64]:
"Description : Create data, model and label folder"
data_path=os.path.join(os.getcwd(),"data")
model_path=os.path.join(os.getcwd(),"model")
minmax_path=os.path.join(os.getcwd(),"minmax")
zip_path=os.path.join(os.getcwd(),"zip")
pyc_model_path=os.path.join(os.getcwd(),"pycmodel")
report_path = os.path.join(os.getcwd(), "reports")
#deleting folder
delete_directory(directorys=[data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

#creating folder
make_directory([data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

In [16]:
def model():
    XG = XGBClassifier()
    return XG

In [17]:
"""
Description : Create architecture of the model
"""
model=model()
print(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)


In [1]:
"""
Description: Trainig the model 
"""

model.fit(X_train,y_train)

'\nDescription: Trainig the model \n'

In [19]:
"""
Saving the model under model directory
"""
modelname = "xgboost_tc_mdl.pkl"
pickle.dump(model, open(model_path+"/"+modelname, 'wb'))

In [20]:
"""
Description: Get prediction on validation data
"""
pred=model.predict(X_test)

In [21]:
print('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(y_test, pred)))
print('F1 score of the Decision Tree model is {}'.format(f1_score(y_test, pred)))
print('Confusion Matrix \n {}'.format(confusion_matrix(y_test, pred)))
print('Classification Report \n {}'.format(metrics.classification_report(y_test, pred)))

Accuracy score of the Decision Tree model is 0.9159949499854326
F1 score of the Decision Tree model is 0.601932811780948
Confusion Matrix 
 [[8778  354]
 [ 511  654]]
Classification Report 
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      9132
           1       0.65      0.56      0.60      1165

    accuracy                           0.92     10297
   macro avg       0.80      0.76      0.78     10297
weighted avg       0.91      0.92      0.91     10297



# 4.0 Prepare Data , Model and MinMax

In [22]:
"""
Description: Zip data
"""
df.to_csv(os.path.join(data_path,"banking_final_dataset.csv"),index=False)
make_archive(base_name=os.path.join(zip_path,"data"),root_dir=data_path,zip_format='zip')

In [23]:
"""Prepare MinMax"""

min_values = df.min().to_numpy()
max_values = df.max().to_numpy()
x= np.array([min_values,max_values])
df_m = pd.DataFrame(x,columns=df.columns)
df_m.to_csv(minmax_path+"/minmax.csv",index=False)

In [24]:
"""
Description: Zip Minmax
"""
make_archive(base_name=os.path.join(zip_path,"minmax"),root_dir=minmax_path,zip_format='zip')

In [25]:
"""
Description: Create encrypted Model
"""
def create_model_pyc(model, model_framework,pyc_model_path):
    
    if model_framework.lower() == "scikit-learn":

        python_code ='''
#importing libraries
import pickle
import zipfile
import os

"""
    class for base model
"""
#define class
class BaseModel():
    def __init__(self,model_path=""):

        """
        constructor for class

        Parameters
        ----------
        input_shape : TYPE, optional
            DESCRIPTION. The default is (100,1).
        Returns
        -------
        None.

        """
        self.model_path = model_path
        
    def load_protected_pickleModel(self,filename,password,picklemodelname):
      

        """
        model architecture

        Parameters
        ----------
        filename : string
            DESCRIPTION.zipped protected filename
            
        password : String
            DESCRIPTION.zipped file password
        
        picklemodelname  :String
            DESCRIPTION.pickle file name present in a ziiped protected
            
        Returns
        -------
        model : model
            DESCRIPTION.

        """
        filepath = os.path.join(self.model_path,filename)
        #print(filepath)
        #print(filename)
        with zipfile.ZipFile(filepath, 'r') as file:
            with file.open(picklemodelname,'r',pwd = bytes(password, 'utf-8')) as f:
                pck = pickle.load(f)
        return pck
    
    def predict(self,X):

        """
        predict for given data

        Parameters
        ----------
        X : numpy array 
            DESCRIPTION.

        Returns
        -------
        pred : numpy array
            DESCRIPTION.

        """
        filename = "modelencrypt.zip"
        password = "987654321"
        picklemodelname = "xgboost_tc_mdl.pkl"
        model = self.load_protected_pickleModel(filename=filename,password=password,picklemodelname=picklemodelname)
        pred = model.predict(X)
        return pred'''
        # Writing to file
        with open("base_model.py", "w") as file:
            # Writing data to a file
            file.writelines(python_code)
         
        """
        Description: function to create .pyc file
        """
        py_compile.compile(file="base_model.py",cfile=pyc_model_path+'/base_model.pyc')
        
        """
        Description: function to create zipped password protected pickle model file
        """

        def zip_model(input_path,output_path,password,com_lvl=5):
            pyminizip.compress(input_path, None, output_path,
                               password, com_lvl)
            
        zip_model(input_path =os.path.join(model_path,modelname),output_path = pyc_model_path+"/modelencrypt.zip" ,password = "987654321",com_lvl=5)
            

In [26]:
"""
Description: Creating encrypted Model
"""
model_framework = "scikit-learn"
model_encryption = 0 #0 for non encrypted file and 1 for encrypted (pyc) 
create_model_pyc(model,model_framework, pyc_model_path)

In [27]:
"""
Description: Zip model
"""
 #0 if model is uploaded directly as a zip, 1 if model is encryted as .pyc and uploaded as a zip
if os.path.isfile(os.path.join(zip_path,"model.zip")):
    delete_directory(directorys=[os.path.join(zip_path,"model.zip")])
if model_encryption:
    make_archive(base_name=os.path.join(zip_path,"bfsi_pyc_model"),root_dir=pyc_model_path,zip_format='zip')
else:
    make_archive(base_name=os.path.join(zip_path,"model"),root_dir=model_path,zip_format='zip')

# 5.0 AIShield API Call

In [65]:
"""
Description: AIShield API URL and subscription key
""" 
baseurl = "XXXXXXXXXXXXXXXXXXXXXXXXXXX" # fill in API endpoint url from AIShield developer portal under API tab 
url = baseurl + "/api/ais/v1.5"
headers={'Cache-Control': 'no-cache',
'x-api-key': "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX", # fill in subscription key from AIShield developer portal under My Dashboard tab
'Org-Id' : "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # fill in Org_Id provided in welcome email
}

In [35]:
"""
Description: call Model registration api to get unique model it and url to upload data, model and label
"""
model_registration_url = url + "/model_registration/upload"
model_registration_payload = {
    'task_type':"TC",
    "analysis_type": "MEA"
}
new_request = requests.request(method="POST", url=model_registration_url, headers=headers, json=model_registration_payload)
new_request=json.loads(new_request.text)
if 'data' in new_request.keys():
    model_id = new_request["data"]['model_id']
    data_upload_url = new_request["data"]["urls"]['data_upload_url']
    label_upload_url = new_request["data"]["urls"]['minmax_upload_url']
    model_upload_url = new_request["data"]["urls"]['model_upload_url']

    print('model_id: ', model_id)
else:
    for k, v in new_request.items():
        print("* {} : {}".format(k,v))

model_id:  38320ed1-8c31-42b4-8b74-c77cc3a10dce


In [37]:
"""
Description: Files path
"""
data_path=os.path.join(zip_path,'data.zip')  # full path of data zip
label_path=os.path.join(zip_path,'minmax.zip') # full path of label zip
# model_path=os.path.join(zip_path,'bfsi_pyc_model.zip') # uncomment if model_encryption = 1 (pyc)
model_path=os.path.join(zip_path,'model.zip') # uncomment if model_encryption = 0 (unencrypted)

In [38]:
def upload_file(url, file_path):
    """
    url: URL to upload
    file_path: file to be uploaded
    """
    new_request = requests.request(method="PUT", url=url, data=open(file_path,'rb'))
    status_cd = new_request.status_code
    if status_cd == 200:
        status = 'upload sucessful'
    else:
        status = 'upload failed'
    return status

In [39]:
"""
Description: Hit AIShield File Upload API
"""
data_upload_status = upload_file(data_upload_url, data_path)
print('data_upload_status: ', data_upload_status)
label_upload_status = upload_file(label_upload_url, label_path)
print('label_upload_status: ', label_upload_status)
model_upload_status = upload_file(model_upload_url, model_path)
print('model_upload_status: ', model_upload_status)


data_upload_status:  upload sucessful
label_upload_status:  upload sucessful
model_upload_status:  upload sucessful


### 5.2 Model Analysis

In [44]:
payload = {
                        "model_details":"NA",
                        "use_model_api":"no",
                        "model_id": model_id,
                        'input_dimensions': "41188,18",
                        "number_of_classes": 2,
                        "number_of_attack_queries": 50000,
                        "vulnerability_threshold": 0,
                        "attack_type": 'blackbox',
                        "encryption_strategy": 0,  # no encryption
                        "normalize_data": "yes",
                        "defense_bestonly": "yes",
                        "is_category_columns" : "yes",
                        "categorical_columns_info" : "job,marital,education,default,housing,loan,contact,poutcome",
                        "model_api_details" :"na",
                        "model_framework":"scikit-learn"
}

In [67]:
"""
Description: Hit AIShield VulnerabilityReport api
"""
model_analysis_url = url + "/model_analyse/"+model_id
if data_upload_status == "upload sucessful" and model_upload_status == "upload sucessful" and label_upload_status == "upload sucessful":
    new_request = requests.request(method="POST", url=model_analysis_url, json=payload,headers=headers)
    new_request=json.loads(new_request.text)
    for k, v in new_request.items():
        print("* {} : {}".format(k,v))


* api_version : 1.5
* job_id : XXXXXXXXXXXXXXXXXXXX
* monitor_link : XXXXXXXXXXXXXXXXXXXXXXXXX


In [68]:
"""
Description: Get job id from api response
"""
job_id=new_request['job_id']
print(f"Job id : {job_id}")

Job id : XXXXXXXXXXXXXXXXXXXX


In [47]:
def monitor_api_progress(new_job_id):
        job_status_url = url + "/job_status_detailed?job_id=" + new_job_id

        # status dictionary
        status_dictionary = {
            'ModelExploration_Status': 'na',
            'SanityCheck_Status': 'na',
            'QueryGenerator_Status': 'na',
            'VunerabilityEngine_Status': 'na',
            'DefenseReport_Status': 'na',
        }
        counts = [0] * len(status_dictionary)
        failed_api_hit_count = 0
        while True:
            time.sleep(2)
            try:
                job_status_response = requests.request("GET", job_status_url, params={},
                                                       headers=headers)

                job_status_payload = json.loads(job_status_response.text)
                failing_key = 'ModelExploration_Status'
                for i, key in enumerate(status_dictionary.keys()):
                    if status_dictionary[key] == 'na':
                        if job_status_payload[key] == 'inprogress' and status_dictionary[key] == 'na':
                            status_dictionary[key] = job_status_payload[key]
                            print(str(key), ":", status_dictionary[key])

                        elif job_status_payload[key] == 'completed' or job_status_payload[key] == 'passed':
                            status_dictionary[key] = job_status_payload[key]
                            counts[i] += 1
                            print(str(key), ":", status_dictionary[key])

                        if job_status_payload[key] == 'failed':
                            failing_key = key
                            status_dictionary[key] = job_status_payload[key]
                            print(str(key), ":", status_dictionary[key])

                    elif job_status_payload[key] == 'completed' or job_status_payload[key] == 'passed':
                        status_dictionary[key] = job_status_payload[key]
                        if counts[i] < 1:
                            print(str(key), ":", status_dictionary[key])
                        counts[i] += 1

                    else:
                        if job_status_payload[key] == 'failed':
                            failing_key = key
                            status_dictionary[key] = job_status_payload[key]
                            print(str(key), ":", status_dictionary[key])

                if job_status_payload[failing_key] == 'failed':
                    break

                if status_dictionary['VunerabilityEngine_Status'] == 'passed' or status_dictionary[
                    'VunerabilityEngine_Status'] == 'completed' and job_status_payload[
                    'CurrentStatus'] == "Defense generation is not triggered":
                    print("\n Vulnerability score {} failed to cross vulnerability threshold of {}".format(
                        job_status_payload['VulnerabiltyScore'], payload['vulnerability_threshold']))
                    break
                if job_status_payload['DefenseReport_Status'] == 'completed':
                    break
            except Exception as e:
                failed_api_hit_count += 1
                print("Error {}. trying {} ...".format(str(e), failed_api_hit_count))
                if failed_api_hit_count >= 3:
                    break
        return status_dictionary

In [48]:
"""
Description: Continuos monitoring of jod progress
"""
status_dictionary = monitor_api_progress(new_job_id=job_id)

ModelExploration_Status : completed
SanityCheck_Status : passed
QueryGenerator_Status : completed
VunerabilityEngine_Status : completed
DefenseReport_Status : completed


In [49]:
def download_artifact(job_id, report_type='Vulnerability', file_format=0):
    """
    job_id: job_id  received after successful api call
    report_type: report to be downloaded
    file_format: change file_format to : 0- all report in zip 
                        1- report in .txt 
                        2- report in .pdf
                        3- report in .json
                        4- report in .xml
    """
    report_url = url + "/get_report?job_id=" + str(
        job_id) + "&report_type=" + report_type + "&file_format=" + str(file_format)

    headers1=headers
    headers1["content-type"]= "application/zip"

    response = requests.request("GET", report_url, params={}, headers=headers1)

    if file_format == 0 or "Attack_samples":
        with open(os.path.join(report_path, report_type + ".zip"), 'wb') as f:
            f.write(response.content)
    elif file_format == 1:
        with open(os.path.join(report_path, report_type + ".txt"), 'wb') as f:
            f.write(response.content)
    elif file_format == 2:
        with open(os.path.join(report_path, report_type + ".pdf"), 'wb') as f:
            f.write(response.content)
    elif file_format == 3:
        with open(os.path.join(report_path, report_type + ".json"), 'wb') as f:
            f.write(response.content)
    elif file_format == 4:
        with open(os.path.join(report_path, report_type + ".xml"), 'wb') as f:
            f.write(response.content)

In [50]:
"""
Description: download generated artifact
"""
if status_dictionary["VunerabilityEngine_Status"] == 'completed':
    download_artifact(job_id=job_id, report_type='Vulnerability', file_format=0) 
    download_artifact(job_id=job_id, report_type='Attack_samples', file_format=0)

if status_dictionary["DefenseReport_Status"] == 'completed':
    download_artifact(job_id=job_id, report_type='Defense', file_format=0)
    download_artifact(job_id=job_id, report_type='Defense_artifact', file_format=0)

In [51]:
def zip_extractor(file, extract_path=None, delete_zip=False):
    """
    extract zip file to the given path

    Parameters
    ----------
    file : path of zip file
    extract_path : path to extract zip file, default considered parent directory
    delete_zip: True, delete zip file after unzipping it

    Returns
    -------
    None.
    """
    if extract_path is None:
        extract_path = os.path.dirname(file)
    print("Extracting : {}".format(file))
    zf = zipfile.ZipFile(file=file, mode='r')
    zf.extractall(extract_path)
    zf.close()
    if delete_zip:
        os.remove(file)
        print("{} removed successfully.".format(file))


In [69]:
"""
Description: Extracting defense artifact
"""
zip_extractor(file=os.path.join(report_path, 'Defense_artifact.zip'))

'\nDescription: Extracting defense artifact\n'

In [70]:
"""
Description: Extracting attack sample
"""
zip_extractor(file=os.path.join(report_path, 'Attack_samples.zip'))

'\nDescription: Extracting attack sample\n'

In [54]:
"""
Description: Load defense model
"""
defense_model_path = os.path.join(report_path, 'defense_model.pkl')
defense_model = pickle.load(open(defense_model_path,'rb'))

In [55]:
"""
Description: Use defense model
"""
from reports import predict
defense = predict.AISDefenseModel(defense_model,model_framework="scikit-learn")

In [56]:
print(defense.defense_model)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [57]:
"""
Description: Pass Original sample data to get prediction
"""
label, prob = defense.predict(X_test[:5])
print("label: {}".format(label))
print("prob: {}".format(prob))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

label: ['not-attack' 'not-attack' 'not-attack' 'not-attack' 'not-attack']
prob: [0 0 0 0 0]


In [62]:
"""
Description: Loading the attack data
"""
attack_sample = pd.read_csv('reports/attack_samples.csv')


In [59]:
"""
Description: Pass attack sample data to get prediction
"""
attack_label, attack_prob= defense.predict(attack_sample[:5])
print("attack_label: {}".format(attack_label))
print("attack_prob: {}".format(attack_prob))


100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

attack_label: ['attack' 'attack' 'attack' 'attack' 'attack']
attack_prob: [1 1 1 1 1]
